<a href="https://colab.research.google.com/github/fbeilstein/machine_learning/blob/master/seminar_5_5_optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
print("Hello")

Hello


Explore gradient descent with the following code.
You should understand how it works and its weaknesses.
If you run out of ideas, use the following parameters

function_name | x_ini | y_ini | max_iter | theta | comment
---|---|---|---|---|---
```x**2 + y**2``` | -5.0 | -4.0 | 20 | 0.1 | good convex function
```sin(x) + sin(y)``` | 1.0 | 1.0 | 10 | 0.01 | many global minima
```(1-x**2)+100*(y-x**2)**2``` | -1.0 | -1.0 | 10 | 0.01 | famous Rosenbrock function

In [2]:
#@title # Exploring simple gradient descent

from plotly.subplots import make_subplots
import plotly.graph_objects as go
import numpy as np

#@markdown ---
#@markdown ##Gradient descent options
#@markdown These variables correspond to the parameters of the minimizator
x_ini = 0 #@param {type:"slider", min:-10.0, max:10.0, step:0.1}

y_ini = 0 #@param {type:"slider", min:-10.0, max:10.0, step:0.1}

max_iter = 50 #@param {type:"slider", min:2, max:50, step:1}

theta = 0.735 #@param {type:"slider", min:0, max:1, step:0.001}
#@markdown ---
#@markdown ##Function to minimize
#@markdown Write function using sympy syntax.
#@markdown You can use all elementary functions, (inverse)trigonometric, (inverse)hyperbolic functions, etc.
#@markdown for more details visit http://www.cfm.brown.edu/people/dobrush/am33/SymPy/function.html

function_name = '(1-x**2)+100*(y-x**2)**2' #@param {type:"string"}

from sympy.parsing.sympy_parser import standard_transformations, implicit_multiplication_application, parse_expr
transformations = (standard_transformations + (implicit_multiplication_application,))
f = parse_expr(function_name, transformations=transformations)

x_min = -1.2
x_max = 1.2
y_min = -1.2
y_max = 1.2

# function calculated for 3d plot
x_ = np.linspace(x_min, x_max, num=50)
y_ = np.linspace(y_min, y_max, num=50)
z_ = np.array([[f.evalf(subs={'x':x__, 'y':y__}) for x__ in x_] for y__ in y_], dtype=float)

from sympy import diff
Gx = diff(f, 'x')
Gy = diff(f, 'y')

trace = [[x_ini, y_ini, f.evalf(subs={'x':x_ini, 'y':y_ini})]]
for i in range(max_iter):
  g_x = Gx.evalf(subs={'x':trace[-1][0], 'y':trace[-1][1]})
  g_y = Gy.evalf(subs={'x':trace[-1][0], 'y':trace[-1][1]})
  x_new = trace[-1][0] - theta * g_x
  y_new = trace[-1][1] - theta * g_y
  z_new = f.evalf(subs={'x':x_new, 'y':y_new})
  trace.append([x_new, y_new, z_new])
trace = np.array(trace, dtype=float)

fig = make_subplots(rows=1, cols=2, specs=[[{"type": "scene"}, {"type": "xy"}]])

fig.add_trace(go.Scatter3d(x=trace[:, 0], y=trace[:, 1], z=trace[:, 2], 
                           marker=dict(size=4, colorscale='Viridis'),
                           line=dict(color='red', width=2)),
              row=1, col=1)

fig.add_trace(go.Surface(x=x_, y=y_, z=z_, opacity=0.9, showscale=False),
              row=1, col=1)

fig.add_trace(go.Contour(z=z_, x=x_, y=y_, contours=dict(showlabels=True)),
              row=1, col=2)

fig.add_trace(go.Scatter(x=trace[:, 0], y=trace[:, 1], line=dict(color='red', width=2)),
              row=1, col=2)

fig.update_layout(width=1200, height=600, autosize=False, 
                  title_text="Gradient descent demonstration",
                  scene=dict(aspectratio = dict(x=1, y=1, z=1)),
                  showlegend=False)
fig.show()

In [1]:
#@title # Exploring minimization methods

from plotly.subplots import make_subplots
import plotly.graph_objects as go
import numpy as np
from scipy.optimize import minimize

#@markdown ---
#@markdown ##Minimization options
#@markdown These variables correspond to the parameters of the minimizator
x_ini = -4.9 #@param {type:"slider", min:-10.0, max:10.0, step:0.1}
y_ini = -3.6 #@param {type:"slider", min:-10.0, max:10.0, step:0.1}


method = 'Newton-CG' #@param ['Nelder-Mead','Powell','CG','BFGS','Newton-CG','L-BFGS-B','TNC','COBYLA','SLSQP','trust-constr','dogleg','trust-ncg','trust-exact','trust-krylov']

max_iter = 10 #@param {type:"slider", min:2, max:50, step:1}

jacobian = 'custom' #@param ['2-point', '3-point', 'cs', 'custom']

#@markdown ---
#@markdown ##Function to minimize

function_name = 'sin(x)*cos(y)' #@param ['x^2 + y^2', 'sin(x)*cos(y)', 'rosen']

from scipy.optimize import rosen, rosen_der

functions = {
    'x^2 + y^2' : lambda x : x[0]**2 + x[1]**2,
    'sin(x)*cos(y)' : lambda x : np.sin(x[0]) * np.cos(x[1]),
    'rosen' : rosen
    }
derivatives = {
    'x^2 + y^2' : lambda x : np.array([2*x[0], 2*x[1]]),
    'sin(x)*cos(y)' : lambda x : np.array([np.cos(x[0]) * np.cos(x[1]), -np.sin(x[0]) * np.sin(x[1])]),
    'rosen' : rosen_der
    }
if jacobian == 'custom':
  jacobian = derivatives[function_name]

func_to_minimize = functions[function_name]
history = [np.array([x_ini, y_ini])]
store_data = lambda xk : history.append(xk)

minimize(func_to_minimize, [x_ini, y_ini], 
         method=method, jac=jacobian, 
         options={'maxiter':max_iter}, callback=store_data)

x_min = -10.0
x_max = 10.0
y_min = -10.0
y_max = 10.0

x_ = np.linspace(x_min, x_max, num=50)
y_ = np.linspace(y_min, y_max, num=50)
z_ = func_to_minimize(np.meshgrid(x_, y_))

_h = np.array(history)
_x = _h.T[0]
_y = _h.T[1]
_z = np.array([func_to_minimize(x) for x in history])

fig = make_subplots(rows=1, cols=2, specs=[[{"type": "scene"}, {"type": "xy"}]])

fig.add_trace(go.Scatter3d(x=_x, y=_y, z=_z, marker=dict(size=4, colorscale='Viridis'),
                                             line=dict(color='red', width=2)),
              row=1, col=1)

fig.add_trace(go.Surface(x=x_, y=y_, z=z_, opacity=0.9, showscale=False),
              row=1, col=1)

fig.add_trace(go.Contour(z=z_, x=x_, y=y_, contours=dict(showlabels=True)),
              row=1, col=2)

fig.add_trace(go.Scatter(x=_x, y=_y, line=dict(color='red', width=2)),
              row=1, col=2)

fig.update_layout(width=600, height=300, autosize=False, 
                  title_text="Gradient descent demonstration",
                  scene=dict(aspectratio = dict(x=1, y=1, z=1)),
                  showlegend=False)
fig.show()